# Neural Network 101
In this example, we will go through the simplest form of a neural network model: a singel input, single output linear function. We expect to predict any used car price based on its mileage using this model.

The following contents will be covered:
1. How to construct a linear model.
2. How to evaluate a model (loss function).
3. How to compute gradient of the loss function.
4. How to update a model.

## Load Data
We will pick 10 samples from the [Used Car Dataset](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data). Each sample contains mileage value and price value of a used car. We need to build a model based on these samples to predict any used car's price based on its mileage.

First, let's load the data as numpy arrays.

In [ ]:
import numpy as np
mileage = np.array([5.8, 7.1, 1.9, 4.1, 12.8, 6.9, 2.9, 4.3, 1.7, 3.0])  # x10,000 miles
price = np.array([3.36, 2.26, 3.96, 3.10, 1.5, 2.8, 3.46, 3.5, 3.0, 3.8])  # x$10,000
print(f"shape of mileage array: {mileage.shape}, shape of price arrary: {price.shape}")  # check dimensions of the arrays

## Visualize the Data
Use [matplotlib](https://matplotlib.org/). **Note**: `matplotlib` will auto scale the figure to better fill the space.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mileage, price, 'bo')
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
# plt.savefig('mile_vs_price-10.png')

## Create an Initial Model
Create a linear model: $y=wx+b$. Initialize two parameters, $w=-1$ for the slope, $b=1$ for the y-axis intersection. **Note:** the figure's scale is different from above, especially on the vertical axis.

In [ ]:
# Create model
def forward(weight, bias, independent_variable):
    dependent_variable = weight * independent_variable + bias
    return dependent_variable
# Visualize the model
w = 1.0
b = -1.0
X = np.linspace(0, 14, 10)  # create 10 evenly distributed independent variables
yhat = forward(w, b, X)  # 10 dependent variables based on initial model
plt.plot(mileage, price, 'bo', X, yhat, 'r')  # plot scattered datapoints as well as the model line
plt.xlim(0., 14.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()
# plt.savefig('init_model.png')

## Create Loss Function
Use Mean Square Error function to evaluate how bad the model was.

In [ ]:
def mse_fn(yhat, y):  # mean square error function
    individual_error = 1 / 2 * (yhat - y) ** 2
    averaged_error = np.mean(individual_error)
    return averaged_error

# Evaluate initial model
pred = forward(w, b, mileage)
print(f"predicted prices: {pred}")
print(f"true prices: {price}")
loss = mse_fn(yhat=pred, y=price)
print(f"mse loss: {loss}")

# Plot errors
err = forward(w, b, mileage) - price
print(f"prediction - true: {err}")
plt.plot(mileage, price, 'bo', X, yhat, 'r')  # plot scattered datapoints as well as the model line
plt.errorbar(mileage, forward(w, b, mileage), yerr=abs(err), uplims=err>0, lolims=err<0, fmt='k.')
plt.xlim(0., 14.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()
# plt.savefig('model_performance.png')

## Gradient Descent Optimization

In [ ]:
# Back propogation
def backward(yhat, y, X):
    dw = np.mean((yhat - y) * X)  # dL/dw
    db = np.mean(yhat - y)  # dL/db
    loss = mse_fn(yhat, y)

    return dw, db, loss

# Optimization
num_iters = 400
learning_rate = 0.05
weights = [w]
biases = [b]
losses = []
for i in range(num_iters):
    pred = forward(w, b, mileage)
    dw, db, mse_loss = backward(pred, price, mileage)
    w = w - learning_rate * dw
    b = b - learning_rate * db
    print(f"loss @ {i+1} iteration: {mse_loss}")
    print(f"w = {w}, b = {b}")
    weights.append(w)
    biases.append(b)
    losses.append(mse_loss)

## Observe the Progress
### Did loss decrease?

In [ ]:
plt.plot(losses)
plt.xlabel("Iterations")
plt.ylabel("MSE Error")
plt.grid()

### How did the model evolve?

In [ ]:
model_0 = yhat
model_1 = forward(weights[1], biases[1], X)
model_2 = forward(weights[2], biases[2], X)
model_10 = forward(weights[10], biases[10], X)
model_100 = forward(weights[100], biases[100], X)
model_200 = forward(weights[200], biases[200], X)
model_400 = forward(weights[400], biases[400], X)
plt.plot(X, model_0, 'red', label='model0')
plt.plot(X, model_1, 'orange', label='model1')
plt.plot(X, model_2, 'yellow', label='model2')
plt.plot(X, model_10, 'green', label='model10')
plt.plot(X, model_100, 'cyan', label='model100')
plt.plot(X, model_200, 'blue', label='model200')
plt.plot(X, model_400, 'purple', label='model400')
plt.plot(mileage, price, 'ko')  # plot scattered datapoints
plt.legend()

### Does the final model fit to the data?

In [ ]:
err_400 = forward(weights[-1], biases[-1], mileage) - price
plt.plot(mileage, price, 'bo', X, model_400, 'r')  # plot scattered datapoints as well as the model line
plt.errorbar(mileage, forward(w, b, mileage), yerr=abs(err_400), uplims=err_400>0, lolims=err_400<0, fmt='k.')
plt.xlim(0., 14.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()

## Make Predictions (Evaluation/Test)

In [ ]:
mileage_test = np.array([0.31, 3.4, 1.9, 1.5 , 3.1, 3.2, 1.2, 0.4, 3.0, 2.3])
price_test = np.array([2.3, 1.8, 3.3, 3.1, 3.4, 2.4, 3.1, 3.5, 2.9, 3.1])
pred_test = forward(weights[-1], biases[-1], mileage_test)
err_test = pred_test - price_test
plt.plot(mileage_test, price_test, 'bo', X, model_400, 'r')  # plot scattered datapoints as well as the model line
plt.errorbar(mileage_test, pred_test, yerr=abs(err_test), uplims=err_test>0, lolims=err_test<0, fmt='k.')
plt.xlim(0., 4.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()
print(f"Model's mse loss on test data: {mse_fn(pred_test, price_test)}")

## Improve Model with More Data

In [ ]:
# Load 1000 data points
mileage_100 = np.load('mileage_100.npy') / 1e4
price_100 = np.load('price_100.npy') / 1e4
# Visualize new data set
X_100 = np.linspace(0, 22, 100)  # create 100 evenly distributed independent variables
yhat_100 = forward(w, b, X_100)  # 100 dependent variables based on final model
plt.plot(mileage_100, price_100, 'bo', X_100, yhat_100, 'r')  # plot scattered datapoints as well as the model line
plt.xlim(0., 22.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()


In [ ]:
# Continue training
num_iters = 200
learning_rate = 0.05
losses_100 = []
for i in range(num_iters):
    pred_100 = forward(w, b, mileage_100)
    dw, db, mse_loss = backward(pred_100, price_100, mileage_100)
    w = w - learning_rate * dw
    b = b - learning_rate * db
    print(f"loss @ {i+1} iteration: {mse_loss}")
    losses_100.append(mse_loss)

In [ ]:
pred_test_new = forward(w, b, mileage_test)
err_test = pred_test_new - price_test
plt.plot(mileage_test, price_test, 'bo', X, forward(w, b, X), 'r')  # plot scattered datapoints as well as the model line
plt.errorbar(mileage_test, pred_test_new, yerr=abs(err_test), uplims=err_test>0, lolims=err_test<0, fmt='k.')
plt.xlim(0., 4.)  # specify range of y axis
plt.xlabel('Mileage (10,000 miles)')
plt.ylabel('Price ($10,000)')
plt.grid()
print(f"New model's mse loss: {mse_fn(pred_test_new, price_test)}")